In [1]:
'''
Author: ChenHJ
Date: 2022-10-27 17:11:38
LastEditors: ChenHJ
LastEditTime: 2022-10-27 17:12:05
FilePath: /chenhj/0302code/cal_uqvq_detrend.ipynb
Aim: 
计算uq,vq,uqvqdiv的detrend和trend
Mission: 
'''
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import Cdo
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof

### 读取non-detrend的uq、vq、uqvqdiv

In [2]:
uqhis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/his_uq_dpg.nc")
vqhis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/his_vq_dpg.nc")

uqvqdivhis_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/non_detrend/his_div_uqvq.nc")

uqssp585_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ssp585_uq_dpg.nc")
vqssp585_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ssp585_vq_dpg.nc")

uqvqdivssp585_ver_JJA = xr.open_dataarray("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/ssp585_div_uqvq.nc")

### 将数据从1979-2099年拼接在一起

In [3]:
uqCMIP6_ver_JJA = xr.concat([uqhis_ver_JJA.sel(time=uqhis_ver_JJA.time.dt.year>=1979), uqssp585_ver_JJA], "time")
vqCMIP6_ver_JJA = xr.concat([vqhis_ver_JJA.sel(time=vqhis_ver_JJA.time.dt.year>=1979), vqssp585_ver_JJA], "time")
uqvqdivCMIP6_ver_JJA = xr.concat([uqvqdivhis_ver_JJA.sel(time=uqvqdivhis_ver_JJA.time.dt.year>=1979), uqvqdivssp585_ver_JJA], "time")

### 计算变量的trend

In [4]:
uqCMIP6_ver_JJA_trend = uqCMIP6_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
vqCMIP6_ver_JJA_trend = vqCMIP6_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)
uqvqdivCMIP6_ver_JJA_trend = uqvqdivCMIP6_ver_JJA.polyfit(dim="time",deg=1,skipna=True)["polyfit_coefficients"].sel(degree=1)

### 输出计算的trend

In [5]:
uqCMIP6_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/uqCMIP6_trend.nc")
vqCMIP6_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/vqCMIP6_trend.nc")
uqvqdivCMIP6_ver_JJA_trend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/non_detrend/uqvqdivCMIP6_trend.nc")

### 计算并输出去趋势的结果

In [6]:
uqCMIP6_ver_JJA_detrend = ca.detrend_dim(uqCMIP6_ver_JJA, "time", deg=1, demean=True)
vqCMIP6_ver_JJA_detrend = ca.detrend_dim(vqCMIP6_ver_JJA, "time", deg=1, demean=True)
uqvqdivCMIP6_ver_JJA_detrend = ca.detrend_dim(uqvqdivCMIP6_ver_JJA, "time", deg=1, demean=True)

uqCMIP6_ver_JJA_detrend.name = "uq_dpg"
vqCMIP6_ver_JJA_detrend.name = "vq_dpg"
uqvqdivCMIP6_ver_JJA_detrend.name = "div_uqvq"

uqhis_ver_JJA_detrend = ca.detrend_dim(uqCMIP6_ver_JJA_detrend.sel(time=uqCMIP6_ver_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
vqhis_ver_JJA_detrend = ca.detrend_dim(vqCMIP6_ver_JJA_detrend.sel(time=vqCMIP6_ver_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)
uqvqdivhis_ver_JJA_detrend = ca.detrend_dim(uqvqdivCMIP6_ver_JJA_detrend.sel(time=uqvqdivCMIP6_ver_JJA_detrend.time.dt.year<=2014), "time", deg=1, demean=True)

uqssp585_ver_JJA_detrend = ca.detrend_dim(uqCMIP6_ver_JJA_detrend.sel(time=uqCMIP6_ver_JJA_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
vqssp585_ver_JJA_detrend = ca.detrend_dim(vqCMIP6_ver_JJA_detrend.sel(time=vqCMIP6_ver_JJA_detrend.time.dt.year>=2015), "time", deg=1, demean=True)
uqvqdivssp585_ver_JJA_detrend = ca.detrend_dim(uqvqdivCMIP6_ver_JJA_detrend.sel(time=uqvqdivCMIP6_ver_JJA_detrend.time.dt.year>=2015), "time", deg=1, demean=True)

uqssp585_p3_ver_JJA_detrend = ca.detrend_dim(uqCMIP6_ver_JJA_detrend.sel(time=uqCMIP6_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
vqssp585_p3_ver_JJA_detrend = ca.detrend_dim(vqCMIP6_ver_JJA_detrend.sel(time=vqCMIP6_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)
uqvqdivssp585_p3_ver_JJA_detrend = ca.detrend_dim(uqvqdivCMIP6_ver_JJA_detrend.sel(time=uqvqdivCMIP6_ver_JJA_detrend.time.dt.year>=2064), "time", deg=1, demean=True)


uqhis_ver_JJA_detrend.name = "uq_dpg"
vqhis_ver_JJA_detrend.name = "vq_dpg"
uqvqdivhis_ver_JJA_detrend.name = "div_uqvq"

uqssp585_ver_JJA_detrend.name = "uq_dpg"
vqssp585_ver_JJA_detrend.name = "vq_dpg"
uqvqdivssp585_ver_JJA_detrend.name = "div_uqvq"

uqssp585_p3_ver_JJA_detrend.name = "uq_dpg"
vqssp585_p3_ver_JJA_detrend.name = "vq_dpg"
uqvqdivssp585_p3_ver_JJA_detrend.name = "div_uqvq"

uqhis_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_uq_dpg.nc")
vqhis_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_vq_dpg.nc")
uqvqdivhis_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_div_uqvq_dpg.nc")

uqssp585_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_uq_dpg.nc")
vqssp585_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_vq_dpg.nc")
uqvqdivssp585_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_div_uqvq_dpg.nc")

uqssp585_p3_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_p3_uq_dpg.nc")
vqssp585_p3_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_p3_vq_dpg.nc")
uqvqdivssp585_p3_ver_JJA_detrend.to_netcdf("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_p3_div_uqvq_dpg.nc")